my known lie doesn't make sense anymore? Well truth is when examples of truth are given (and maybe it's asked to tell the truth?) I would need to analyze this seperatly, but getting say N tokens. And trying differen't combinatiuons of system prompt, and n-shot examples


In [ ]:
import torch
from IPython.display import display, HTML, Markdown
from torch.utils.data import random_split, DataLoader, TensorDataset
from loguru import logger
import sys
import pandas as pd
import seaborn as sns


logger.remove()
logger.add(sys.stderr, format="{time} {level} {message}", level="INFO")

# load my code
%load_ext autoreload
%autoreload 2
from src.eval.collect import manual_collect2
from src.eval.ds import ds2df, qc_ds, qc_dsdf
from src.prompts.prompt_loading import load_prompts, format_prompt, load_preproc_dataset
from src.llms.load import load_model

from matplotlib import pyplot as plt
plt.style.use(['seaborn-v0_8'])


In [ ]:
from pathlib import Path
BASE_FOLDER = Path("/media/wassname/SGIronWolf/projects5/elk/sgd_probes_are_lie_detectors/notebooks/lightning_logs/version_6/")

## Importance matrix

In [ ]:
from src.probes.importance_matrix import get_importance_matrix

f = f"{BASE_FOLDER}/checkpoint_last/adapter_model.safetensors"
importance_matrix = get_importance_matrix(f, layers=['fc1', 'Wqkv'])#[SKIP+1::STRIDE, ::DECIMATE]
print(importance_matrix.shape)
plt.hist(importance_matrix.flatten(), bins=55);


## Load DS

In [ ]:
from datasets import Dataset

f1_val = next(iter(BASE_FOLDER.glob('hidden_states/.ds/ds_valtest_*')))
f1_ood = next(iter(BASE_FOLDER.glob('hidden_states/.ds/ds_OOD_*')))
f1_val, f1_ood

ds_val = Dataset.from_file(str(f1_val)).with_format("torch")
ds_oos = Dataset.from_file(str(f1_ood)).with_format("torch")

# ds_known1 = filter_ds_to_known(ds_out, verbose=True)

In [ ]:
qc_ds(ds_val)
qc_ds(ds_oos)


In [ ]:
def agg_func(x):
    acc = (x.label_instructed == x["ans"]).mean()
    return pd.Series(dict(acc=acc, n=len(x)))

df = ds2df(ds_val)
df = df.rename(columns=lambda x: x.replace("_base", "")).copy()

for col in ["sys_instr_name", "ds_string"]:
    display(df.groupby(col).apply(agg_func))


# Probe

In [ ]:
from src.probes.lr import TorchRobustScaler, TorchLogisticRegression, TorchDummyClassifier
from src.helpers.ds import train_test_split_ds
from src.eval.pl_sk import analyze_dfres, SKEvaluator, PlainTruthEval

### LR

In [ ]:
print('logistic regression')
evaluator = PlainTruthEval(ds_trainval=ds_val, ds_test=ds_oos, importance_matrix=importance_matrix, skip=1, stride=1, decimate=1)
model = TorchLogisticRegression(random_state=42, 
                                max_iter=10,
                                penalty='elasticnet',
                                # tol=1e-2,
                                solver='saga',
                                l1_ratio=0.5,
                                 class_weight='balanced',)
df_res = evaluator.eval(model)
insample_datasets = list(set(ds_val['ds_string_base']))
analyze_dfres(df_res, insample_datasets, insample_datasets)

In [ ]:
print('DummyClassifier')
baseline_model = TorchDummyClassifier(random_state=42, strategy="most_frequent")
df_res2 = evaluator.eval(baseline_model)
insample_datasets = list(set(ds_val['ds_string_base']))
analyze_dfres(df_res2, insample_datasets, insample_datasets)

# Other models

In [ ]:
from src.probes.direct_bce_ranking import PLSK_BCE, PLSKWrapper
from src.eval.pl_sk import RankingTruthEval, DistTruthEval, RankingObeyEval, PlainObeyEval
import numpy as np


max_epochs = 10
verbose = True
batch_size=16

In [ ]:
# Unit test
for cls in [RankingTruthEval, DistTruthEval, RankingObeyEval, PlainObeyEval]:
    evaluator = cls(ds_trainval=ds_val, ds_test=ds_oos, importance_matrix=importance_matrix)
    proxy = evaluator.ds2proxy(ds_val)
    y1 = evaluator.proxy2label(proxy, ds_val)
    np.testing.assert_array_equal(y1, ds_val['label_true_base'], err_msg="undo_distance_truth_telling failed")

### Binary cross entropy

In [ ]:
print('Binary cross entropy')
evaluator = RankingTruthEval(ds_trainval=ds_val, ds_test=ds_oos, importance_matrix=importance_matrix, skip=1, stride=1, decimate=1)
steps_per_epoch = int(len(ds_val)*0.7//batch_size)

_X, _ = evaluator.ds2xy(ds_val)
c_in = _X.shape[1:]


pl_model = PLSK_BCE(c_in=c_in, steps_per_epoch=steps_per_epoch, max_epochs=max_epochs, dropout=0.25, hs=16, depth=3)
model = PLSKWrapper(pl_model, verbose=verbose, max_epochs=max_epochs)
df_res = evaluator.eval(model)
insample_datasets = list(set(ds_val['ds_string_base']))
analyze_dfres(df_res, insample_datasets, insample_datasets)

### MSE Ranking

In [ ]:
from src.probes.mse_ranking import PLSKMSErank

print('MSE Ranking')
evaluator = RankingTruthEval(ds_trainval=ds_val, ds_test=ds_oos, importance_matrix=importance_matrix, skip=1, stride=1, decimate=1)
steps_per_epoch = int(len(ds_val)*0.7//batch_size)

_X, _ = evaluator.ds2xy(ds_val)
c_in = _X.shape[1:]


pl_model = PLSKMSErank(c_in=c_in, steps_per_epoch=steps_per_epoch, max_epochs=max_epochs, dropout=0.25, hs=16, depth=3)
model = PLSKWrapper(pl_model, verbose=verbose, max_epochs=max_epochs)
df_res = evaluator.eval(model)
insample_datasets = list(set(ds_val['ds_string_base']))
analyze_dfres(df_res, insample_datasets, insample_datasets)

### Bool ranking

In [ ]:
from src.probes.mse_ranking import PLSKBoolRank

print('MSE Ranking')
evaluator = RankingTruthEval(ds_trainval=ds_val, ds_test=ds_oos, importance_matrix=importance_matrix, skip=1, stride=1, decimate=1)
steps_per_epoch = int(len(ds_val)*0.7//batch_size)

_X, _ = evaluator.ds2xy(ds_val)
c_in = _X.shape[1:]


pl_model = PLSKBoolRank(c_in=c_in, steps_per_epoch=steps_per_epoch, max_epochs=max_epochs, dropout=0.25, hs=16, depth=3)
model = PLSKWrapper(pl_model, verbose=verbose, max_epochs=max_epochs)
df_res = evaluator.eval(model)
insample_datasets = list(set(ds_val['ds_string_base']))
analyze_dfres(df_res, insample_datasets, insample_datasets)

### CCS

In [ ]:
from src.probes.ccs import PLSKCCS

In [ ]:

print('CCS')
evaluator = RankingTruthEval(ds_trainval=ds_val, ds_test=ds_oos, importance_matrix=importance_matrix, skip=1, stride=1, decimate=1)
steps_per_epoch = int(len(ds_val)*0.7//batch_size)

_X, _ = evaluator.ds2xy(ds_val)
c_in = _X.shape[1:]


pl_model = PLSKCCS(c_in=c_in, steps_per_epoch=steps_per_epoch, max_epochs=max_epochs, dropout=0.25, hs=16, depth=3)
model = PLSKWrapper(pl_model, verbose=verbose, max_epochs=max_epochs)
df_res = evaluator.eval(model)
insample_datasets = list(set(ds_val['ds_string_base']))
analyze_dfres(df_res, insample_datasets, insample_datasets)

: 

### ConformalLR

In [ ]:
from src.probes.conformal import MapieClassifier2
from sklearn.linear_model import LogisticRegression

print('ConformalLR')
evaluator = RankingTruthEval(ds_trainval=ds_val, ds_test=ds_oos, importance_matrix=importance_matrix, skip=1, stride=1, decimate=1)

clf = LogisticRegression(random_state=42, max_iter=1000, class_weight='balanced',)
model = MapieClassifier2(estimator=clf, 
                        # cv="prefit",
                        method="score",                        
                        )
df_res = evaluator.eval(model)
insample_datasets = list(set(ds_val['ds_string_base']))
analyze_dfres(df_res, insample_datasets, insample_datasets)

# Compare importance matrix

In [ ]:
from sklearn.metrics import roc_auc_score, accuracy_score
from einops import rearrange
import safetensors.torch

def get_importance_matrix2(saved_adaptop_file, layers=['fc1', 'Wqkv']):
    state_dict = safetensors.torch.load_file(saved_adaptop_file)
    keys = sorted(state_dict.keys())
    if layers is None:
        layers = list(set([k.split('.')[-2] for k in state_dict.keys()]))

    activations = {}

    for k in keys:
        suffix = k.split('.')[-2]
        if suffix not in activations:
            activations[suffix] = []
        activations[suffix].append(state_dict[k])

    for k in activations.keys():
        activations[k] = rearrange(activations[k], 'l h b -> (b l) h').detach().cpu().float()
        print(k, activations[k].shape)


    importance_matrix = torch.concat([activations[i] for i in layers], dim=1)
    return importance_matrix

In [ ]:
print('no matrix')
evaluator = PlainTruthEval(ds_trainval=ds_val, ds_test=ds_oos, 
                        #    importance_matrix=importance_matrix
                           skip=1, stride=1, decimate=1
                           )
model = TorchLogisticRegression(random_state=42, 
                                penalty='elasticnet',
                                solver='saga',
                                l1_ratio=0.5,
                                 class_weight='balanced',)
df_res = evaluator.eval(model)
insample_datasets = list(set(ds_val['ds_string_base']))
df_res_ana = analyze_dfres(df_res, insample_datasets, insample_datasets)

In [ ]:
print('matrix')
evaluator = PlainTruthEval(ds_trainval=ds_val, ds_test=ds_oos, 
                           importance_matrix=importance_matrix
                           , skip=1, stride=1, decimate=1
                           )
model = TorchLogisticRegression(random_state=42, 
                                penalty='elasticnet',
                                solver='saga',
                                l1_ratio=0.5,
                                 class_weight='balanced',)
df_res = evaluator.eval(model)
insample_datasets = list(set(ds_val['ds_string_base']))
analyze_dfres(df_res, insample_datasets, insample_datasets)
roc_auc_score(df_res['y'], df_res['y_prob'])

In [ ]:
class PlainTruthEvalTop(PlainTruthEval):
    """only take the top X % of importance values."""

    def ds2xy(self, ds: Dataset):
        """function which will transform a dataset into x and y."""
        data = []
        for layer in self.layers_names:
            # Stack the base and adapter representations as a 4th dim
            X1 = [ds[f'end_residual_{layer}_base'], ds[f'end_residual_{layer}_adapt']]
            X1 = rearrange(X1, 'versions b l f  -> b l f versions')
            data.append(X1)
        
        # concat layers
        # x = rearrange(data, 'parts b l f v -> b l (parts f) v')
        x = torch.concat(data, dim=2)
        
        # restrict to non zero values
        mask = self.importance_matrix>1
        mask = rearrange(mask, 'l f -> (l f)')
        xf = rearrange(x, 'b l f v -> b (l f) v')[:, mask, :]
        print(f'keeping top {(mask*1.0).mean():2.2%} of values {x.shape} -> {xf.shape}')
        x = rearrange(xf, 'b lf v -> b lf 1 v') 
        y = self.ds2proxy(ds)
        return x, y


In [ ]:
print('top 10%')

importance_matrix = get_importance_matrix2(f, layers=['fc1', 'Wqkv'])
importance_matrix = (importance_matrix-1).abs()
importance_matrix = importance_matrix / importance_matrix.std()
importance_matrix = importance_matrix + 1
importance_matrix = importance_matrix * ((importance_matrix>4))

# plt.hist(importance_matrix.flatten(), bins=55)
# plt.show();

evaluator = PlainTruthEvalTop(ds_trainval=ds_val, ds_test=ds_oos, 
                           importance_matrix=importance_matrix, skip=1, stride=1, decimate=1
                           )
model = TorchLogisticRegression(random_state=42, 
                                penalty='elasticnet',
                                solver='saga',
                                # max_iter=100,
                                l1_ratio=0.5,
                                 class_weight='balanced',)
df_res = evaluator.eval(model)
insample_datasets = list(set(ds_val['ds_string_base']))
analyze_dfres(df_res, insample_datasets, insample_datasets)